## Tokenization

### 1. 基于词典的分词

分词思路：
- 准备常用词字典
- 从文本开头，尽可能从字典中匹配最长的词
- 如果匹配不到，则从开头匹配最短的词
- 每次匹配到一个词，就将该词从文本中删除

伪代码：
1. 初始化一个词典（可以根据你的任务需求动态扩展）。
2. 从文本的第一个字开始，尝试匹配尽可能长的子串。
3. 如果匹配成功，将该词从文本中移除，然后继续处理剩余文本。
4. 如果匹配失败，则减少匹配长度，直到找到一个匹配的词，或回退到一个字进行匹配。
5. 重复此过程直到文本被完全处理。

数据集
- THUCNews - 文本分类数据集 - http://thuctc.thunlp.org/
- THUOCL (Tsinghua University Open Chinese Lexicon) 词库每一行由两部分组成，分别是词和DF值（存在此单词的文档个数），中间由Tab间隔。


#### Step-0: 从THUOCL中加载常用词字典


In [3]:
word_category = 'animal'
word_dict = set()
# 读取文件
with open('../datasets/THUOCL-master/data/THUOCL_'+word_category+'.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

for line in lines[:]:
    word_dict.add(line.strip().split('\t')[0])
print(word_dict)

{'越南猫', '肩背银鳞蛛', '秘鲁拾叶雀', '端褐长跗萤叶甲', '达特穆尔马', '脊髓动物', '大骨顶', '斑颊泞鰕虎鱼', '短臀中波鱼', '纯种比熊犬', '棕星脚', '山字宽盾蝽', '詹鱼', '圆斑伟黄斑蜂', '畑井氏歌津鱼龙', '眼斑帆鳍杜父鱼', '匙吻苏禄油鲶', '奇汉西喷雾蟾蜍', '锥纹光壳蛤', '亚马孙河放电小眼鱼', '油炸蚂蚱', '跗跖骨', '古董核螺', '黑胁啄花鸟', '彗星副狮子鱼', '姣长体茧蜂', '黄眶离缘姬蜂', '波纹扭月贝属', '驼背棘鮟鱇', '神农吻鰕虎鱼', '秀唇旋虫', '二甲氧苄氨嘧啶', '粘虫棘领姬蜂', '玻利维亚凤凰', '眼圈熊', '圆翅紫斑蝶', '玫瑰花蓝水母', '黑尾叶蝉头蝇', '约旦墨头鱼', '台湾缨口鳅', '中间角鮃', '三带椭龟甲', '泉龙', '云斑俊盲蝽', '奥粪金龟', '褐斑筍螺', '灰缝叶莺', '粗唇鮠', '母猪壳', '一点突额杆蝇', '长吻锯尾鲨', '苏卡迪芋螺', '日本医蛭', '普通楼燕', '近缘柱螯磷虾多形手磷虾', '斑点真海精戎', '钮灰蝶属', '金口蠑螺', '金盲眼灯鱼', '鱼胶原蛋白', '幕骨', '中间角鲆', '始祖单弓兽属', '石苇', '西里汉梗', '叶腕水母', '真胃穴水母', '叶形狭啮', '中华窄腹剑水蚤', '新畿内亚孔唇鳗鲶', '新几内亚下鱵鱼', '宠物营养师', '阿坝革蜱', '茶肖叶甲', '科尔沁牛', '黑眉辉蜂鸟', '卢卡斯梗犬', '贝诺虫属', '玉珠粗皮鬘螺', '列齿鳗光唇鳗', '陆良褐菌瓢虫', '大西洋驼海豚', '杜比黑顶康吉鳗', '科斯勒宝螺', '榆蘑', '纽芬兰刺狮子鱼', '犬只饲养', '亚利桑那大麻哈鱼', '矮鮨朴丽鱼', '三道眉拟雀', '维切格达鳄属', '煤山雀', '褐翅燕鸥', '自实丰仓归', '棕翅唐加拉雀', '大头糯鳗', '小型秘鲁无毛犬', '结肠小袋纤毛虫', '百慕大鱵', '牛霸鹟', '谷仓猫头鹰', '石灰虫', '梅岭秃角蝉', '莫那龙', '日本紫云尺蛾', '矛尾复鰕虎鱼', '小褐果鸠', '花曲柳窄吉丁', '英国斗牛犬', '日本四星步行虫', '钝齿斯氏萤',

In [1]:
from typing import List, Set


# from dict_import import dict_import


def dict_import(dict_list: List[str],file_path:str) -> Set[str]:
    word_dict = set()
    for dict_category in dict_list:
        with open(file_path +'THUOCL_'+ dict_category +'.txt', 'r', encoding='utf-8') as file:
            lines = file.readlines()
        for line in lines:
            word_dict.add(line.strip().split('\t')[0])
    return word_dict


dict_path = '../datasets/THUOCL-master/data/'
# dict_categories = ['animal', 'caijing','car','chengyu','diming','food','IT','law','lishimingren','medical', 'poem']
dict_categories = ['animal']
dict_all = dict_import(dict_categories, dict_path)

# user_input = ''
token_result = ''
user_input = input("请输入待分词中文段: ")

p_start = 0
p_end = 0
while p_start < len(user_input):
    p_end = len(user_input)
    while p_end > p_start:
        if user_input[p_start:p_end] in dict_all:
            token_result += user_input[p_start:p_end] + '/'
            break
        else:
            p_end -= 1
    p_start = p_end
# 输出一定的信息
print(f"分词结果：{token_result}")

KeyboardInterrupt: 

### 2. 基于统计的分词

In [4]:
# 接受用户输入
user_input = input("请输入您的名字: ")

# 输出一定的信息
print(f"您好，{user_input}！欢迎使用 Python！")


您好，哈哈哈哈哈哈哈哈哈哈哈！欢迎使用 Python！
